In [45]:
import pandas as pd
import numpy as np

In [46]:
df_listino = pd.read_csv('data_demo/art_costi.csv')
df_ddt = pd.read_csv('data_demo/ddt_righe.csv')

df_ddt = df_ddt[df_ddt["Data_documento"] > "2024-02-18"]
# remove from listino al the products that are not in the ddt
df_listino = df_listino[df_listino["Codice_articolo"].isin(df_ddt["Codice_articolo"])]

In [47]:
df_listino.head()

,Codice_articolo,Codice_variante,Descrizione,Costo,Data_riferimento,Categoria
26,017.PM152,1,VETRO STRATIFICATO 33.1,30.5,2023-08-28,Vetri
27,017.PM152,2,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri
28,017.PM152,3,VETRO STRATIFICATO 33.1,30.5,2023-08-28,Vetri
29,017.PM152,4,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri
30,017.PM152,5,VETRO STRATIFICATO 33.1,18.0,2023-08-28,Vetri


In [48]:
dict_entities = {}
ids_products_list = []
ids_clienti_list = []
ids_azien_list = []

In [49]:
# prodotti in listino
for index in range(len(df_listino)):
    cod_articolo = df_listino.iloc[index]["Codice_articolo"]
    ids_products_list.append(cod_articolo)
    variante = df_listino.iloc[index]["Codice_variante"]
    costo = df_listino.iloc[index]["Costo"]
    data_riferimento = df_listino.iloc[index]["Data_riferimento"]
    categoria = df_listino.iloc[index]["Categoria"]
    descrizione = df_listino.iloc[index]["Descrizione"]
    dict_entities[cod_articolo] = {
        "cod_articolo": "prod_"+cod_articolo,
        "variante": variante,
        "costo": costo,
        "data_riferimento": data_riferimento,
        "categoria": categoria,
        "descrizione": descrizione
    }

In [50]:
# prodotti inventati
for index in range(20):
    cod_articolo = "prod_"+str(index)
    ids_products_list.append(cod_articolo)
    variante = np.random.choice([0, 1, 2])
    costo = np.random.uniform(10000, 20000)
    data_riferimento = "2024-03-17"
    categoria = "level2"
    descrizione = "prodotto inventato"
    dict_entities[cod_articolo] = {
        "cod_articolo": cod_articolo,
        "variante": variante,
        "costo": costo,
        "data_riferimento": data_riferimento,
        "categoria": categoria,
        "descrizione": descrizione
    }

In [51]:
#fornitori e ddt
for index in range(len(df_ddt)):
    cod_fornitore = df_ddt.iloc[index]["COD_CF_Fornitore"]
    dict_entities["fornitore_" + cod_fornitore] = {
        "cod_fornitore": cod_fornitore
    }
    data_documento = df_ddt.iloc[index]["Data_documento"]
    id_documento = df_ddt.iloc[index]["ID_documento"]
    descrizione = df_ddt.iloc[index]["Descrizione"]
    codice_articolo = df_ddt.iloc[index]["Codice_articolo"]
    codice_variante = df_ddt.iloc[index]["Codice_variante"]
    codice_categoria = df_ddt.iloc[index]["Codice_categoria"]
    unita_misura = df_ddt.iloc[index]["Unita_misura"]
    quantita_um = df_ddt.iloc[index]["Quantita_um"]
    prezzo_netto_vu = df_ddt.iloc[index]["Prezzo_netto_vu"]
    importo = df_ddt.iloc[index]["importo"]

    if id_documento not in dict_entities:
        dict_entities["documento_" + id_documento] = {
            "data_documento": data_documento,
            "id_documento": id_documento,
            "venduto": [],
            "importo": importo
        }
    dict_entities["documento_" + id_documento]["venduto"].append(
        {
        "codice_articolo": codice_articolo,
        "codice_variante": codice_variante,
        "descrizione": descrizione,
        "codice_categoria": codice_categoria,
        "unita_misura": unita_misura,
        "quantita_um": quantita_um,
        "prezzo_netto_vu": prezzo_netto_vu,
        }
    )

In [52]:
#clienti
for i in range(60):
    dict_entities["cliente_"+str(i)] = { "id_cliente": "cliente_"+str(i)}
    ids_clienti_list.append("cliente_"+str(i))

In [53]:
# aziende
for i in range(3):
    dict_entities["mangini_"+str(i)] = { "id_azienda": "mangini_"+str(i)}
    ids_azien_list.append("mangini_"+str(i))

In [54]:
# preventivi
for i in range(35):
    data_documento =  "2024-03-17"
    id_documento = "preventivo_"+str(i)
    descrizione = "preventivo inventato"
    id_cliente = np.random.choice(ids_clienti_list)
    id_azienda = np.random.choice(ids_azien_list)
    products_list = []
    for j in range(np.random.randint(1, 5)):
        # chose a random product
        codice_articolo = np.random.choice(ids_products_list)
        product = dict_entities[codice_articolo]
        codice_variante = product["variante"]
        costo = product["costo"]
        categoria = product["categoria"]
        prezzo_netto_vu = costo + np.random.uniform(-0.25, 0.5)
        product_dict = {
            "codice_articolo": codice_articolo,
            "codice_variante": codice_variante,
            "codice_categoria": categoria,
            "prezzo_netto_vu": prezzo_netto_vu,
        }
        products_list.append(product_dict)

    importo = sum([product["prezzo_netto_vu"] for product in products_list])

    dict_entities[id_documento] = {
        "data_documento": data_documento,
        "id_documento": id_documento,
        "da_vendere": products_list,
        "importo": importo,
        "id_cliente": id_cliente,
        "id_azienda": id_azienda
        }

In [55]:
# fatture
for i in range(70):
    data_documento =  "2024-03-17"
    id_documento = "fattura_"+str(i)
    descrizione = "fattura inventata"
    id_cliente = np.random.choice(ids_clienti_list)
    id_azienda = np.random.choice(ids_azien_list)
    products_list = []
    for j in range(np.random.randint(1, 5)):
        # chose a random product
        codice_articolo = np.random.choice(ids_products_list)
        product = dict_entities[codice_articolo]
        codice_variante = product["variante"]
        costo = product["costo"]
        categoria = product["categoria"]
        prezzo_netto_vu = costo + np.random.uniform(-0.25, 0.5)
        product_dict = {
            "codice_articolo": codice_articolo,
            "codice_variante": codice_variante,
            "codice_categoria": categoria,
            "prezzo_netto_vu": prezzo_netto_vu,
        }
        products_list.append(product_dict)

    importo = sum([product["prezzo_netto_vu"] for product in products_list])

    dict_entities[id_documento] = {
        "data_documento": data_documento,
        "id_documento": id_documento,
        "venduto": products_list,
        "importo": importo,
        "id_cliente": id_cliente,
        "id_azienda": id_azienda
        }

In [56]:
dict_entities

{'017.PM152': {'cod_articolo': '017.PM152',
  'variante': 0,
  'costo': 10000.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO 33.1'},
 '090.VST33.SI': {'cod_articolo': '090.VST33.SI',
  'variante': 56,
  'costo': 33.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE 33.1 SILENT F.G. ANTA INT.'},
 '090.VST33': {'cod_articolo': '090.VST33',
  'variante': 0,
  'costo': 24.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE STRATIFICATO 3+3 F.G. ANTA INTELAIATA'},
 '090.VST55': {'cod_articolo': '090.VST55',
  'variante': 0,
  'costo': 37.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO TRASPARENTE 5+5'},
 '090.VST66.SI': {'cod_articolo': '090.VST66.SI',
  'variante': 0,
  'costo': 51.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE 66.1 ANTIRUMOR